# Python для анализа данных

## Работа с форматами XML и JSON

# XML

`До этого мы с вами собирали данные вручную, обращаясь к html страницам, размеченным для отображения в браузере. Но данные также можно собирать и через API -  — application program interface. Обычный интерфейс — это способ взаимодействия человека с программой, а API — одной программы с другой. Например, вашего скрипта на Python с удалённым веб-сервером.

Для хранения веб-страниц, которые читают люди, используется язык HTML. Для хранения произвольных структурированных данных, которыми обмениваются между собой программы, используются другие языки — в частности, язык XML, похожий на HTML. Вернее было бы сказать, что XML это метаязык, то есть способ описания языков. В отличие от HTML, набор тегов в XML-документе может быть произвольным (и определяется разработчиком конкретного диалекта XML). Например, если бы мы хотели описать в виде XML некоторую студенческую группу, это могло бы выглядеть так:

```xml
<group>
    <number>134</number>
    <student>
        <firstname>Виталий</firstname>
        <lastname>Иванов</lastname>
    </student>
    <student>
        <firstname>Мария</firstname>
        <lastname>Петрова</lastname>
    </student>
</group>
```

Для обработки XML-файлов можно использовать тот же пакет *Beautiful Soup*, который мы уже использовали для работы с HTML. Единственное различие — нужно указать дополнительный параметр `feautres="xml"` при вызове функции `BeautifulSoup` — чтобы он не искал в документе HTML-теги.

In [1]:
group = """<group>
<number>134</number>
<student>
<firstname>Виталий</firstname>
<lastname>Иванов</lastname>
</student>
<student>
<firstname>Мария</firstname>
<lastname>Петрова</lastname>
</student>
</group>"""

In [2]:
!pip install lxml

In [3]:
from bs4 import BeautifulSoup

obj = BeautifulSoup(group, features="lxml")
print(obj.prettify())

<html>
 <body>
  <group>
   <number>
    134
   </number>
   <student>
    <firstname>
     Виталий
    </firstname>
    <lastname>
     Иванов
    </lastname>
   </student>
   <student>
    <firstname>
     Мария
    </firstname>
    <lastname>
     Петрова
    </lastname>
   </student>
  </group>
 </body>
</html>


Номер группы можно найти, например, вот так - для каждого объекта через точку указываем его атрибут, в который надо спуститься.

In [6]:
obj.group.number.text # последний атрибут текст, точно также как делали в html

'134'

Но это работает только тогда, когда тэг уникальный. В других случаях, парсер всегда будет попадать в первый child-тэг, который он встретил по пути вниз.

In [7]:
obj.group.student.lastname.text # до Петровой так не добраться

'Иванов'

Перечислить всех студентов можно с помощью цикла (похожая структура у нас была и в обработке html).

In [8]:
for student in obj.group.find_all('student'):
    print(student.lastname.text, student.firstname.text)

Иванов Виталий
Петрова Мария


По сути, главное отличие xml от html, что работать вы будете не со стандартизированными структурами. Поэтому перед работой придется поиграть в детективов - запросить данные и внимательно изучить расположение узлов, чтобы понять, какие тэги вас интересуют.

XML легко представить в виде дерева, где есть главный узел (parent) и его "дети".

![](https://www.py4e.com/images/xml-tree.svg)  
*Источник: Python for Everybody, C.Severance*

Кроме BS парсить xml можно и с помощью других библиотек. Например, ElementTree.


In [9]:
import xml.etree.ElementTree as ET

In [10]:
tree = ET.fromstring(group)
list(tree) # посмотрим, что внутри

[<Element 'number' at 0x78a4c822f970>,
 <Element 'student' at 0x78a4c822f9c0>,
 <Element 'student' at 0x78a4c822fab0>]

Cинтаксис очень похож на BS. Добрались до первой фамилии.

In [11]:
tree.find('student').find('lastname').text

'Иванов'

In [12]:
for element in tree.findall('student'):
    print(element)
    print(element.find('lastname').text)

<Element 'student' at 0x78a4c822f9c0>
Иванов
<Element 'student' at 0x78a4c822fab0>
Петрова


Можно немного упростить код, включив дочерний тэг в findall.

In [13]:
for element in tree.findall('student/lastname'):
    print(element.text)

Иванов
Петрова


# Задача
По ссылке данные в формате xml.
http://py4e-data.dr-chuck.net/comments_42.xml

Посчитайте все комментарии в этом документе (поля count). Попробуйте сделать это с помощью list comprehension.

In [ ]:
# TODO

# JSON

Другой популярный формат, в котором клиент может отдать вам данные - json. JSON расшифровывается как JavaScript Object Notation и изначально возник как подмножество языка JavaScript (пусть вас не вводит в заблуждение название, этот язык ничего не имеет общего с Java), используемое для описания объектов, но впоследствии стал использоваться и в других языках программирования, включая Python. Различные API могут поддерживать либо XML, либо JSON, либо и то, и другое, так что нам полезно научиться работать с обоими типами данных (например, wiki api могла бы выгрузить нам данные и в формате json при соответствующем запросе).

JSON очень похож на описание объекта в Python и смысл квадратных и фигурных скобок такой же. Правда, есть и отличия: например, в Python одинарные и двойные кавычки ничем не отличаются, а в JSON можно использовать только двойные. Мы видим, что полученный нами JSON представляет собой словарь, значения которого — строки или числа, а также списки или словари, значения которых в свою очередь также могут быть строками, числами, списками, словарями и т.д. То есть получается такая довольно сложная структура данных.

В данный момент тот факт, что перед нами сложная структура данных, видим только мы — с точки зрения Python, j.text это просто такая строка. Однако в модуле requests есть метод, позволяющий сразу выдать питоновский объект (словарь или список), если результат запроса возвращён в формате JSON. Так что нам не придётся использовать никакие дополнительные библиотеки.

Преимущества JSON в том, что мы получаем готовый объект Python и нет необходимости использовать какие-то дополнительные библиотеки для того, чтобы с ним работать. Недостатком является то же самое: зачастую поиск информации в XML-файле может проводиться более эффективно, чем в JSON.

## JSON (парсинг VK)

Как уже говорилось выше, не все API открытые. Так, чтобы достать информацию из vk вам придется сгенерировать токен с помощью вашего аккаунта (мы выложим отдельный блокнот как это сделать, для тех, кому будет интересно). API VK отдает данные в json. Структура тут будет посложнее, чем то, что мы уже видели, поэтому давайте еще потренируемся.

Теперь научимся еще и загружать JSON файл с диска. Для этого нам понадобится модуль json.

In [ ]:
import json
with open('vk.json',  'r', encoding='Utf-8') as json_data:
    res_loaded = json.load(json_data) # считываем данные с помощью функции .load()

Здесь у нас выгрузка постов со стены группы ВШЭ.

In [ ]:
res_loaded

{'count': 1542,
 'items': [{'id': 32494,
   'from_id': -132,
   'owner_id': -132,
   'date': 1541422799,
   'marked_as_ads': 0,
   'post_type': 'post',
   'text': '',
   'attachments': [{'type': 'link',
     'link': {'url': 'http://family.hse.ru/event/view/2271',
      'title': 'День карьеры факультета бизнеса и менеджмента',
      'caption': 'family.hse.ru',
      'description': '',
      'photo': {'id': 456239093,
       'album_id': -2,
       'owner_id': 100,
       'photo_75': 'https://pp.userapi.com/c850336/v850336548/628ea/TwPjzdADVZ4.jpg',
       'photo_130': 'https://pp.userapi.com/c850336/v850336548/628eb/gMSUSljxiC8.jpg',
       'photo_604': 'https://pp.userapi.com/c850336/v850336548/628ec/kIMEO_f_3JQ.jpg',
       'width': 150,
       'height': 80,
       'text': '',
       'date': 1541422799}}}],
   'post_source': {'type': 'vk'},
   'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
   'likes': {'count': 6, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
  

In [ ]:
res_loaded['items'][2]

{'id': 32492,
 'from_id': -132,
 'owner_id': -132,
 'date': 1540743239,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'attachments': [{'type': 'link',
   'link': {'url': 'https://logistics.hse.ru/news/226897419.html',
    'title': 'Лучшие выпускники Школы логистики получили международные сертификаты',
    'caption': 'logistics.hse.ru',
    'description': 'Событие в Школе логистики',
    'photo': {'id': 456239301,
     'album_id': -2,
     'owner_id': 100,
     'photo_75': 'https://pp.userapi.com/c846417/v846417451/111745/gC-IAV8kVks.jpg',
     'photo_130': 'https://pp.userapi.com/c846417/v846417451/111746/oKMb8a5d_88.jpg',
     'photo_604': 'https://pp.userapi.com/c846417/v846417451/111747/QD0TSfXe9dI.jpg',
     'width': 150,
     'height': 100,
     'text': '',
     'date': 1540742411}}}],
 'post_source': {'type': 'vk'},
 'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
 'likes': {'count': 5, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
 'reposts': {'

In [ ]:
res_loaded.keys()

dict_keys(['count', 'items'])

Ключами являются `count` и `items`. Нужные нам объекты (текст постов, id автора, дата и время публикации и проч.) находятся в `items`.

In [ ]:
res_loaded['items'][50] # первый элемент items - первый пост со всей информацией о нем

{'id': 32099,
 'from_id': -132,
 'owner_id': -132,
 'date': 1531760787,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'copy_history': [{'id': 1050401,
   'owner_id': -79138567,
   'from_id': -79138567,
   'date': 1531760410,
   'post_type': 'post',
   'text': 'В России есть проблемы с образованием, но часть из них можно решить с помощью грамотного менеджмента.\n\nМагистерская программа НИУ ВШЭ «Экономика и управление образованием» сделает из вас крутого специалиста. Вы научитесь руководить большими образовательными организациями и открывать собственные: школы, курсы, университеты и так далее. \n\nЗадавайте в Прямой линии свои вопросы о поступлении, будущей профессии и образовательном менеджменте:\n\nhttps://thequestion.ru/interview/5663/magistratura-niu-vshe-ekonomika-i-upravlenie-obrazovaniem#',
   'attachments': [{'type': 'doc',
     'doc': {'id': 469433958,
      'owner_id': 27546726,
      'title': 'school3.gif',
      'size': 378724,
      'ext': 'gif',
      'url': 'ht

Помимо текста поста можно найти много всего интересного. Например, тип поста (`post_type`), дата (`date`), id поста (`id`), лайки (`likes`, которые включают информацию о том, могут ли пользователи лайкать пост и публиковать его, а также собственно число лайков), репосты (`reposts`, которые включают число репостов), число просмотров (`views`), комментарии (`comments`, которые включают информацию о том, могут ли пользователи комментировать пост, и число комментариев), и так далее.

Давайте остановимся на тексте поста, id автора, id поста и дате публикации. Чтобы извлечь соответствующую информацию, сохраним `items` и извлечем из них нужные поля:

In [ ]:
items = res_loaded['items']
full_list = []

for item in items:
    l = [item['from_id'], item['id'], item['text'], item['date']] # нужные поля
    full_list.append(l) # добавляем в список списков full_list
# несколько элементов списка
full_list[0:4]

[[-132, 32494, '', 1541422799],
 [-132,
  32493,
  'На Шаболовке 7 ноября в 18:30 ауд 5215 состоится встреча с представителем оргкомитета симпозиума, Lars John. Авторы лучших эссе получат возможность участвовать в симпозиуме в Швейцарии, а автор cамого лучшего эссе получит значительный денежный приз. Приглашаем всех студентов магистратуры, студентов 4 курса и аспирантов принять участие в этой встрече. \n \nTo all grad/postgrad and 4th year undergraduate students: compete until 1 Feb 2019 and qualify as a Leader of Tomorrow for the #49sgs (8–10 May 2019) in Switzerland, all expenses paid. Create an impact and win CHF 20,000. Leaders who have attended in the past include Niall Ferguson, Jack Ma, Christine Lagarde, Paul Polman, Anders Fogh Rasmussen, Ratan Tata, Muhammad Yunus, and many more. Register now at www.symp.sg/apply',
  1541076555],
 [-132, 32492, '', 1540743239],
 [-132, 32491, '', 1540551399]]

Видно, что в двух первых постах текста не обнаружено, там только картинки, ссылки и репосты.

Из этого списка списков можно легко сделать датафрейм `pandas`.

Но если у ссылок, репостов и картинок нет текста, то наш DataFrame будет выглядет неполноценно. Поэтому добавим условие, что если текст отсутсвует, вставим в DataFrame строку с этой отметкой.

In [ ]:
# опять выберем только нужные поля
full_list = []
for item in items:
    if item['text'] == "":
        l = [item['from_id'], item['id'], "Картинка, ссылка или репост", item['date']]
    else:
        l = [item['from_id'], item['id'], item['text'], item['date']]
    full_list.append(l)

In [ ]:
full_list[1]

[-132,
 32493,
 'На Шаболовке 7 ноября в 18:30 ауд 5215 состоится встреча с представителем оргкомитета симпозиума, Lars John. Авторы лучших эссе получат возможность участвовать в симпозиуме в Швейцарии, а автор cамого лучшего эссе получит значительный денежный приз. Приглашаем всех студентов магистратуры, студентов 4 курса и аспирантов принять участие в этой встрече. \n \nTo all grad/postgrad and 4th year undergraduate students: compete until 1 Feb 2019 and qualify as a Leader of Tomorrow for the #49sgs (8–10 May 2019) in Switzerland, all expenses paid. Create an impact and win CHF 20,000. Leaders who have attended in the past include Niall Ferguson, Jack Ma, Christine Lagarde, Paul Polman, Anders Fogh Rasmussen, Ratan Tata, Muhammad Yunus, and many more. Register now at www.symp.sg/apply',
 1541076555]